# Import repo

In [2]:
!git clone https://github.com/FernandoDoreto/convert-streamlit-to-django.git

Cloning into 'convert-streamlit-to-django'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (424/424), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 424 (delta 195), reused 287 (delta 86), pack-reused 0
Receiving objects: 100% (424/424), 373.74 KiB | 4.56 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [2]:
%cd /content/convert-streamlit-to-django

/content/convert-streamlit-to-django


In [3]:
pwd

'/content/convert-streamlit-to-django'

# Load dataset

In [5]:
pip install streamlit

     |████████████████████████████████| 8.2MB 4.3MB/s 
     |████████████████████████████████| 163kB 47.1MB/s 
     |████████████████████████████████| 112kB 51.9MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 4.2MB 52.0MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 122kB 52.8MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=1c5ebbe7d25b2732a46ffbe887e5187b721edcd2b80791b50402ffaeaf448131
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [4]:
from src.processing.data_management import LoadIrisDataset

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [5]:
df = LoadIrisDataset()

* Train Test split

In [6]:
from app_pages.page_ml_ClfIrisSpecies import ML_ClfIrisSpeciesBody
from config import config
from src.processing.data_management import TrainTestSplit
from src.machine_learning.train_pipeline import TrainPipeline_ClfIrisSpecies
import src.machine_learning.pipeline as pipeline

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [7]:
 X_train, X_test,y_train, y_test = TrainTestSplit(df=df,
                                                  TARGET=config.ClfIrisSpecies_TARGET)

# Grid Search on 1 pipeline

In [44]:
from config import config
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 


ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier())),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier())
    ]
)


from sklearn.model_selection import GridSearchCV
_parameters = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


_pipe = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters, 
		cv = 5,   
		n_jobs = -2,
		verbose = 2
		)

In [62]:
_pipe.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feat_selection',
                                        SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                         class_weight=None,
                                                                                         criterion='gini',
                                                                                         max_depth=None,
                                                                                         max_features=None,
                                                                                         max_leaf_nodes=None,
                                                                                         min_impurity_decrease=0.0,
                                                                                         min_impurity_s

In [63]:
_pipe.best_estimator_

Pipeline(memory=None,
         steps=[('feat_selection',
                 SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                  class_weight=None,
                                                                  criterion='gini',
                                                                  max_depth=None,
                                                                  max_features=None,
                                                                  max_leaf_nodes=None,
                                                                  min_impurity_decrease=0.0,
                                                                  min_impurity_split=None,
                                                                  min_samples_leaf=1,
                                                                  min_samples_split=2,
                                                                  min_weight_fract

# Grid Search on Stacking pipeline

In [8]:
from config import config
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV


ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier())),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier())
    ]
)


_parameters_DT = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


PipelineClfIrisSpecies_DT = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters_DT, 
		cv = 5,   
		n_jobs = -2,
		verbose = 2
		)

In [9]:
ClfIrisSpecies_GB = Pipeline(
    [       
        ("feat_selection",SelectFromModel(GradientBoostingClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", GradientBoostingClassifier(random_state=config.RANDOM_STATE,learning_rate=0.1))
    ]
)

_parameters_GB = {
    'model__learning_rate': [1e-1,1e-2,5e-1],
    'model__n_estimators': [100,800]
}


PipelineClfIrisSpecies_GB = GridSearchCV(
		estimator = ClfIrisSpecies_GB,
		param_grid = _parameters_GB, 
		cv = 5,   
		n_jobs = -2,
		verbose = 2
		)

In [10]:
ClfIrisStacking = StackingClassifier(
    classifiers=[
        PipelineClfIrisSpecies_GB,
        PipelineClfIrisSpecies_DT
    ],
    use_probas=True,
    meta_classifier=LogisticRegression(solver='liblinear')
)


In [11]:
ClfIrisStacking.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.4s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.4s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.4s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.4s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.4s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.3s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  

...

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10

[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:    0.5s finished


StackingClassifier(average_probas=False,
                   classifiers=[GridSearchCV(cv=5, error_score=nan,
                                             estimator=Pipeline(memory=None,
                                                                steps=[('feat_selection',
                                                                        SelectFromModel(estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                                                             criterion='friedman_mse',
                                                                                                                             init=None,
                                                                                                                             learning_rate=0.1,
                                                                                                                             loss='deviance',
           

In [13]:
ClfIrisStacking.predict_proba(X_test)

array([[0.9352621 , 0.03260976, 0.03212814],
       [0.03224065, 0.93560376, 0.03215559],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.03222091, 0.03727086, 0.93050823],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.03224065, 0.93560376, 0.03215559],
       [0.03222075, 0.03721328, 0.93056597],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.03224065, 0.93560376, 0.03215559],
       [0.03222075, 0.03721328, 0.93056597],
       [0.03224065, 0.93560376, 0.03215559],
       [0.03224162, 0.93560219, 0.03215618],
       [0.03222075, 0.03721328, 0.93056597],
       [0.03224065, 0.93560376, 0.03215559],
       [0.03222066, 0.03721328, 0.93056606],
       [0.03222095, 0.03721925, 0.9305598 ],
       [0.03224065, 0.93560376, 0.03215559],
       [0.0322408 , 0.93560352, 0.03215568],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.9352621 , 0.03260976, 0.03212814],
       [0.03222244, 0.03726239, 0.93051517],
       [0.